In [73]:
TEST_DIR = '../test/'

# Preprocess route data

In [124]:
import fiona
from fiona.crs import from_epsg
from shapely.geometry import LineString, shape, mapping
import rtree
from shapely.ops import snap
import copy

In [125]:
def snap_linestrings(r, lines):
    # create an empty spatial index object
    index = rtree.index.Index()
    
    snapped = []
    
    # populate the spatial index
    for index_id, geom in enumerate(lines):
        index.insert(index_id, geom.bounds)
    
    # create snapped lines
    for search_id, geom in enumerate(lines):
        
        # buffer the line
        buffered = geom.buffer(r)

        # get list of fids where bounding boxes intersect
        index_ids = [int(i) for i in index.intersection(buffered.bounds)]

        # access the features that those fids reference
        new_geom = copy.deepcopy(geom)
        for index_id in index_ids:
            
            if search_id == index_id:
                continue
            
            new_geom = snap(new_geom, lines[index_id], r)
            
            index.delete(search_id, geom.bounds)
        
        snapped.append(new_geom)
    
    return snapped

In [117]:
def write_shape(geometries, path):
    schema = {'geometry': 'LineString', 'properties': {}}
    with fiona.open(path, 'w', crs=from_epsg(4326), driver='ESRI Shapefile', schema=schema) as output:
        for g in geometries:
          # attributes
          attributes = {}
          # write the row (geometry + attributes in GeoJSON format)
          output.write({'geometry': mapping(g), 'properties': attributes})

In [118]:
# Load road and rail shapefiles
ROAD_PATH = TEST_DIR + 'ne_10m_roads_test.shp'
RAIL_PATH = TEST_DIR + 'ne_10m_railroads_test.shp'
road = [LineString(shape(f['geometry'])) for fin fiona.open(ROAD_PATH)]
rail = [LineString(shape(f['geometry'])) for f in fiona.open(RAIL_PATH)]

# Create new snapped LineString collection
SNAPPED_FILE_NAME = 'snapped.shp'
snapped = snap_linestrings(0.005, road)
write_shape(snapped, TEST_DIR + SNAPPED_FILE_NAME)


# Preprocess livestock and crop data

In [129]:
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform

In [135]:
# Load livestock raster
LIVESTOCK_PATH = TEST_DIR + 'cgiar_glb_cattle_cc2006_ad_test.tif'

# Read raster
with rasterio.open(LIVESTOCK_PATH) as r:
    T0 = r.transform  # upper-left pixel corner affine transform
    p1 = Proj(r.crs)
    A = r.read()  # pixel values

# All rows and columns into numpy mesh grid
cols, rows = np.meshgrid(np.arange(A.shape[2]), np.arange(A.shape[1]))

# Get affine transform for pixel centres
T1 = T0 * Affine.translation(0.5, 0.5)
# Function to convert pixel row/column index (from 0) to easting/northing at centre
rc2en = lambda r, c: (c, r) * T1

# All eastings and northings (there is probably a faster way to do this)
eastings, northings = np.vectorize(rc2en, otypes=[np.float, np.float])(rows, cols)

# Project all longitudes, latitudes
# p2 = Proj(proj='latlong',datum='WGS84')
# longs, lats = transform(p1, p2, eastings, northings)

In [136]:
print(T1)

| 0.01, 0.00, 143.14|
| 0.00,-0.01,-38.99|
| 0.00, 0.00, 1.00|


# Convert to graph and run analysis

In [119]:
import networkx as nx

In [120]:
Groad = nx.read_shp(TEST_DIR + SNAPPED_FILE_NAME)

nx.write_shp(Groad, TEST_DIR)

In [122]:
Groad.nodes()

NodeView(((148.24732818407236, -41.51390395148999), (148.0534803398829, -42.05420326189036), (146.65117678617202, -41.52627721814038), (146.8161536748439, -42.02945672858958), (145.83454118724626, -41.047844240991964), (145.53758278763692, -42.058327684107155), (147.48843449618175, -41.93459501760326), (147.14198302997082, -41.47678415153882), (146.45732894198258, -41.249940929615), (147.1708539854884, -41.571645862525145), (147.443065851797, -41.839733306616935), (147.3069599186427, -42.89558539411689), (147.0512457412013, -43.03169132727118), (145.25712207689475, -40.812752174634554), (146.74191407494155, -41.09321288537673), (147.13373418553724, -41.44378877380445), (146.96875729686536, -43.43176028230046), (146.88626885252944, -41.11795941867751), (147.39769720741222, -41.01897328547439), (147.83901038460948, -41.10971057424391), (147.51730545169934, -41.159203640845476), (146.82440251927747, -41.060217507642356), (147.5585496738673, -42.78422599426338), (147.84313480682627, -43.14

In [123]:
try:
    n = nx.shortest_path_length(G,(148.24732818407236, -41.51390395148999),(147.1708539854884, -41.571645862525145))
    print(n)
except nx.NetworkXNoPath:
    print('No path')

8
